In [68]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [69]:
trainingData = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
    )
testData = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
    )

In [70]:
batch_size = 64

train_dataLoader = DataLoader(trainingData,batch_size = batch_size)
test_dataLoader = DataLoader(testData,batch_size = batch_size)

for x,y in test_dataLoader:
    print(x.shape)
    print(y.shape," , ",y.dtype)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])  ,  torch.int64


In [71]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [72]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [73]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-3)

In [74]:
def train(DataLoader,model,loss_fn,optimizer):
    size = len(DataLoader.dataset)
    model.train()
    for batch ,(x,y) in enumerate(DataLoader):
        x,y = x.to(device),y.to(device)
        
        pred = model(x)
        loss = loss_fn(pred,y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [75]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [76]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataLoader, model, loss_fn, optimizer)
    test(test_dataLoader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306854  [    0/60000]
loss: 2.292969  [ 6400/60000]
loss: 2.275049  [12800/60000]
loss: 2.261025  [19200/60000]
loss: 2.253226  [25600/60000]
loss: 2.216004  [32000/60000]
loss: 2.222739  [38400/60000]
loss: 2.184742  [44800/60000]
loss: 2.191636  [51200/60000]
loss: 2.147508  [57600/60000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 2.148400 

Epoch 2
-------------------------------
loss: 2.168214  [    0/60000]
loss: 2.150800  [ 6400/60000]
loss: 2.094244  [12800/60000]
loss: 2.104933  [19200/60000]
loss: 2.049285  [25600/60000]
loss: 1.981951  [32000/60000]
loss: 2.013394  [38400/60000]
loss: 1.926859  [44800/60000]
loss: 1.945045  [51200/60000]
loss: 1.857068  [57600/60000]
Test Error: 
 Accuracy: 54.3%, Avg loss: 1.863637 

Epoch 3
-------------------------------
loss: 1.906063  [    0/60000]
loss: 1.865096  [ 6400/60000]
loss: 1.751563  [12800/60000]
loss: 1.792614  [19200/60000]
loss: 1.675360  [25600/60000]
loss: 1.627421  [32000/600

In [77]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [78]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [80]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = testData[0][0], testData[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
